In [89]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import scipy.stats as ss

In [90]:
# input data

t0 = '1/9/2023'
t1 = '1/12/2023'
t2 = '1/9/2025'
t3 = '1/14/2025'
f1 = 0.915067
S =  11108.45 
q = .01032
f2 = 0.915013036


In [91]:
dates = list((t0, t1, t2, t3))

In [92]:
dates = [pd.to_datetime(i) for i in dates]

In [93]:
delta = dates[2] - dates[0]
delta

Timedelta('731 days 00:00:00')

In [94]:
T = delta.days/365
T

2.0027397260273974

In [95]:
r = -np.log(f1)/T
r

0.04431828619074147

In [96]:
def bsf(O, S, K, r, sigma, q, T, disFac):
    
    O = str(O)
    
    d1 = (np.log(S/K) + (r - q + (0.5 * sigma**2)) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S/K) + (r - q - (0.5 * sigma**2)) * T) / (sigma * np.sqrt(T))
    
    if "Call" in O:
        result = disFac * (S * np.exp((r - q)*T) * ss.norm.cdf(d1, 0,1) - K * ss.norm.cdf(d2, 0, 1))
    elif "Put" in O:
        result = disFac * (K * ss.norm.cdf(-d2, 0, 1) - S * np.exp((r - q)*T) * ss.norm.cdf(-d1, 0,1))
    elif "Dp" in O:
        result = disFac * (ss.norm.cdf(-d2, 0, 1))
    else:
        result = disFac * (-ss.norm.cdf(d2, 0, 1))
    
        
    return result

In [97]:
bsf("Put", S, S, r, .20, q, T, f2)

859.9541046603317

In [98]:
opt = ['Put', 'Call', 'Put', 'Dp', 'Put']
strikes = [ 12996.89, 12996.89, 11108.45,  8886.76,  8886.76 ]
impl_vol = [0.2551568, 0.2551568, 0.2813162, 0.2924907, 0.2924907]

In [99]:
df = pd.DataFrame({'O': opt, 'K': strikes, 'sigma': impl_vol, 'V': np.zeros(5)})
df

,O,K,sigma,V
0,Put,12996.89,0.255157,0.0
1,Call,12996.89,0.255157,0.0
2,Put,11108.45,0.281316,0.0
3,Dp,8886.76,0.292491,0.0
4,Put,8886.76,0.292491,0.0


In [100]:
df.assign(V = lambda df: bsf(df.O,  S, df.K, r, df.sigma, q, T, f2))

,O,K,sigma,V
0,Put,12996.89,0.255157,1174.172614
1,Call,12996.89,0.255157,1174.172614
2,Put,11108.45,0.281316,2042.245213
3,Dp,8886.76,0.292491,3295.341097
4,Put,8886.76,0.292491,3295.341097


**the values calculated for 'V' are all for Call option as shown below**

In [101]:
df.assign(V = lambda df: bsf("Call",  S, df.K, r, df.sigma, q, T, f2))

,O,K,sigma,V
0,Put,12996.89,0.255157,1174.172614
1,Call,12996.89,0.255157,1174.172614
2,Put,11108.45,0.281316,2042.245213
3,Dp,8886.76,0.292491,3295.341097
4,Put,8886.76,0.292491,3295.341097


In [102]:
df.assign(V = lambda df: df.O)

,O,K,sigma,V
0,Put,12996.89,0.255157,Put
1,Call,12996.89,0.255157,Call
2,Put,11108.45,0.281316,Put
3,Dp,8886.76,0.292491,Dp
4,Put,8886.76,0.292491,Put


**as I find from above that the function is calling df.O properly, but when I use it in the formula, it is only considering Call for all the rows**